In [ ]:
'''
01_download-data.ipynb
'''

import requests
from bs4 import BeautifulSoup
import urllib.request
import pathlib
import pathlib
import os
import shutil
import gdal
import subprocess

import multiprocessing
from joblib import Parallel, delayed
num_cores = multiprocessing.cpu_count()

PATH_ROOT = os.path.join(pathlib.Path().absolute(), '../..' )
PATH_DATA = PATH_ROOT + '/data/01_raw/'

# links
LINK_NE_RIVERS = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_rivers_lake_centerlines_scale_rank.zip"
LINK_NE_RIVERS_EU = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_rivers_europe.zip"

LINK_SRTM_DIR = 'https://dds.cr.usgs.gov/srtm/version2_1/SRTM3/Eurasia/'

DIR_OUT_SRTM  = PATH_DATA + '/srtm/'

In [ ]:
'''
Download google spreadsheet tables
'''

LINK_ACTIVITIES = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSKbKoMl477qDAiyhj9sT9TZlGC7OGzShbDAaELXgkPSR95_CkmZsWmN3vLGsJgEBDkiLdn4Be_mkBJ/pub?gid=0&single=true&output=csv'
LINK_LOCALITIES = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSKbKoMl477qDAiyhj9sT9TZlGC7OGzShbDAaELXgkPSR95_CkmZsWmN3vLGsJgEBDkiLdn4Be_mkBJ/pub?gid=121203183&single=true&output=csv'

urllib.request.urlretrieve(LINK_ACTIVITIES, PATH_DATA + 'activities.csv')
urllib.request.urlretrieve(LINK_LOCALITIES, PATH_DATA + 'localities.csv')


In [ ]:
# Download and merge srtm 

LINK_SRTM1 = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/topo/downloads/GMTED/Global_tiles_GMTED/075darcsec/med/E000/30N000E_20101117_gmted_med075.tif'
LINK_SRTM2 = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/topo/downloads/GMTED/Global_tiles_GMTED/075darcsec/med/E000/50N000E_20101117_gmted_med075.tif'

urllib.request.urlretrieve(LINK_SRTM1, DIR_OUT_SRTM + 'srtm1.tif')
urllib.request.urlretrieve(LINK_SRTM2, DIR_OUT_SRTM + 'srtm2.tif')

In [ ]:
# all remaining srtm files
source_srtm = [DIR_OUT_SRTM + file for file in os.listdir(DIR_OUT_SRTM)]

# merge srtm files
merge_command = ["gdal_merge.py", "-o", DIR_OUT_SRTM + "merged.tif"] + source_srtm
subprocess.check_output(merge_command)